# 日记

* 2019.03.08

>1. 完成整体架构
>2. 实现plotly可视化(candlesticks)
>3. 完成倍量柱
>4. 完成N天M次倍量

* 2019.03.15

>1. 完成阳胜柱(已不用)
>2. 完成将军柱、黄金柱
>3. 完成N天M次黄金

* 2019.03.22

>1. N天K线
>2. 汇出100支股片的2/3天K线csv

* 2019.03.23

>1. 实现OHLC(实现zoom in)
>2. 实现缩量群

In [1]:
import os
import time
import numpy as np
import tushare as ts
import pandas as pd
from datetime import datetime, timedelta
import plotly.offline as py
import plotly.graph_objs as go
from plotly.graph_objs import Scatter, Layout
py.init_notebook_mode(connected=True)

In [2]:
pro = ts.pro_api('3cc122330a310b1088913c4c112b890bcfa12315764d11f867f2bddd')

# 获取最近N天每一支股票的数据

In [140]:
def get_Ndays(today_date,N):
    start = time.time()
    data = []
    date = today_date
    for _ in range(N):
        today = pro.daily(trade_date=date)
        print('【%s】'%date,end='')
        while len(today)==0:
            date = datetime.strftime(pd.to_datetime(date) - timedelta(1), '%Y%m%d')
            today = pro.daily(trade_date=date)
        today['state'] = today['close']>=today['open'] # 阳柱:True, 阴柱:False
        today.name = date
        data.append(today)
        date = datetime.strftime(pd.to_datetime(date) - timedelta(1), '%Y%m%d')
    print('\n获取数据共花了 %s 秒'%(round(time.time()-start,2)))
    return data

# 筛选倍量柱

In [4]:
def get_bl(today, yesterday, v_pct_min=0.9, v_pct_max=1.1, c_pct_min=1.01, ignore=[]):
    print('正在筛选倍量柱...\n')
    start = time.time()
    result = []
    for i in today['ts_code']:
        if i in list(yesterday['ts_code']) and i not in ignore:
            today_v = float(today[today['ts_code']==i]['vol'])
            today_c = float(today[today['ts_code']==i]['close'])
            yesterday_v = float(yesterday[yesterday['ts_code']==i]['vol'])
            yesterday_c = float(yesterday[yesterday['ts_code']==i]['close'])
            v_pct = today_v/yesterday_v
            c_pct = today_c/yesterday_c
            cond1 = v_pct > v_pct_min and v_pct < v_pct_max
            cond2 = c_pct > c_pct_min
            if cond1 and cond2:
                print('{}\tv:{}\tc:{}'.format(i,round(v_pct,2),round(c_pct,2)))
                result.append(i)
    print('\n遍历 %s 支股票共花了 %s 秒'%(len(today),round(time.time()-start,0)))
    return result

# 筛选将军柱/黄金柱

# 将军柱（价平量平）（量柱和价柱对比形成小平台）

1. 【阳胜柱】：所谓阳胜柱，就是量柱和价柱双向胜过其左侧最近的阴柱。

>a. 阳柱收盘高于阴柱开盘

>b. 阳柱的量高于阴柱的量

2. 【收盘价三日不破底】：不破实底就是基柱后三日的收盘价，没有跌破基柱的实底（开盘价）。这里有两种情况：

>a. 基柱后三日的收盘价，均高于或等于基柱实底（开盘价）

>b. 基柱后三日的收盘价平均值，不低于基柱实底（开盘价）

3. 【量柱群三日不过头】两种情况：

>a. 基柱后三日的量柱应逐步缩小（有时包括基柱）

>b. 基柱后三日的量柱可参差不齐，但量价之比相对持平。

# 黄金柱（价升量缩）（量柱和价柱对比形成喇叭口）

1. 【将军柱】

2. 【收盘价三日不破顶】：不破实顶就是基柱后三日的收盘价，没有跌破基柱实顶（收盘价）。这有两种情况：

>a. 基柱后三日的收盘价，均高于或等于基柱实顶（收盘价）

>b. 基柱后三日的收盘价平均值，不低于基柱实顶（收盘价）

3. 【量柱群三日不抬头】

>a. 基柱后三日的量柱应逐步缩小（有时包括基柱）

>b. 基柱后三日的量柱可参差不齐，但量价之比相对加大。

4. 【先者优先】

[5.2：伏击涨停的王牌之将军柱的三个标准](https://www.guminwangxiao.com/jiqiao/fjzt/1043.html)

[5.3：伏击涨停的王牌之黄金柱的三个标准](https://www.guminwangxiao.com/jiqiao/fjzt/1044.html)

In [5]:
def yang(code,date):
    past = get_one(code, date, period=10, print_mode=False)
    this_day = past.tail(1)
    if this_day['state'].item():  # 如果是阳柱
        last = past[past['state'] == False].tail(1)  # 前一个阴柱
        n = 10
        while len(last) == 0:     # 如果往前10天都没有阴柱
            past = get_one(code, date, period=n, print_mode=False)
            last = past[past['state'] == False].tail(1)
            n += 10
        cond1 = this_day['close'].item() > last['open'].item()     # 阳柱收盘高于阴柱开盘
        cond2 = this_day['vol'].item() > last['vol'].item()        # 阳柱的量高于阴柱的量
        if cond1 & cond2:
            return True
        else:
            return False
    else:
        return False

In [6]:
def get_hj(d1, d2, d3, ignore=[]):   # d1→d2→d3
    print('正在筛选黄金柱...\n')
    start = time.time()
    result = []
    for i in d1['ts_code']:
        if i in list(d2['ts_code']) and i in list(d3['ts_code']) and i not in ignore:
            #if yang(i,d1.name):  # 阳胜柱
            if True:  # 不管阳胜柱
                o1 = d1[d1['ts_code']==i]['open'].item()
                c1,c2,c3 = d1[d1['ts_code']==i]['close'].item(),d2[d2['ts_code']==i]['close'].item(),d3[d3['ts_code']==i]['close'].item()
                v1,v2,v3 = d1[d1['ts_code']==i]['vol'].item(),d2[d2['ts_code']==i]['vol'].item(),d3[d3['ts_code']==i]['vol'].item()
                cond_open = o1 < min(c2,c3)        # 收盘价三日不破底
                cond_close = c1 < min(c2,c3)       # 收盘价三日不破顶
                cond_vol = (v1 > v2) & (v2 > v3)   # 量柱群三日不过头、不抬头
                if cond_open and cond_close and cond_vol:
                    print(i)
                    result.append(i)
    print('\n遍历 %s 支股票共花了 %s 秒 找到 %s 支股票符合条件'%(len(d1),round(time.time()-start,0),len(result)))
    return result

# 缩量群：量缩价增

In [119]:
def get_suo(days, rough=0.99, ignore=[]):   # days[0]→days[1]→days[2]
    print('正在筛选 %s 天缩量群...\n'%len(days))
    start = time.time()
    result = []
    result = []
    for code in days[0]['ts_code']:
        In = code not in ignore
        for day in days:
            if not In:
                break
            else:
                In *= code in list(day['ts_code'])
        if not In:
            break
        else:
            price_increase = True
            vol_decrease = True
            for i in range(len(days)-1):
                if price_increase:
                    price_increase *= days[i+1][days[i+1]['ts_code']==code]['close'].item()>days[i][days[i]['ts_code']==code]['close'].item()*rough
                else:
                    break
                if vol_decrease:
                    vol_decrease *= days[i+1][days[i+1]['ts_code']==code]['vol'].item()<days[i][days[i]['ts_code']==code]['vol'].item()/rough
                else:
                    break
            if price_increase and vol_decrease:
                print(code)
                result.append(code)
            else:
                pass            
    print('\n遍历 %s 支股票共花了 %s 秒 找到 %s 支股票符合条件'%(len(d1),round(time.time()-start,0),len(result)))
    return result

# 获取最近N天单只个股的数据

In [107]:
def get_one(code, today_date, period, print_mode=True):
    start_date = datetime.strftime(pd.to_datetime(today_date) - timedelta(period-1), '%Y%m%d')
    name = pro.stock_basic(exchange='', list_status='L', fields='ts_code,name')
    start = time.time()
    df = pro.daily(ts_code=code, start_date=start_date, end_date=today_date).iloc[::-1]
    df['state'] = df['close']>=df['open'] # 阳柱:True, 阴柱:False
    df.name = '%s-%s'%(name[name['ts_code']==code]['name'].item(),code)
    if print_mode:
        print('获取 %s 的 %s 天数据 共花了 %s 秒'%(code, period, round(time.time()-start, 2)))
    return df

In [139]:
get_one('000001.SZ','20190312',10)

获取 000001.SZ 的 10 天数据 共花了 0.51 秒


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,state
6,000001.SZ,20190304,12.70,13.38,12.62,12.99,12.76,0.23,1.8025,2456830.39,3195011.702,True
5,000001.SZ,20190305,12.91,13.29,12.84,13.06,12.99,0.07,0.5389,1408422.22,1838251.656,True
4,000001.SZ,20190306,13.06,13.15,12.84,13.08,13.06,0.02,0.1531,1245760.20,1614670.974,True
3,000001.SZ,20190307,13.06,13.06,12.55,12.74,13.08,-0.34,-2.5994,1782057.31,2274151.007,False
2,000001.SZ,20190308,12.49,12.70,12.25,12.30,12.74,-0.44,-3.4537,1778162.10,2214572.416,False
1,000001.SZ,20190311,12.30,12.49,12.05,12.32,12.30,0.02,0.1626,1157650.77,1416000.379,True
0,000001.SZ,20190312,12.49,12.64,12.24,12.36,12.32,0.04,0.3247,1411794.92,1757797.250,False


# 画图 candlestick

In [8]:
def plotly_candlestick(code,date,period=300):
    
    df = get_one(code, date, period, print_mode=False)
    
    INCREASING_COLOR = '#e2002a'
    DECREASING_COLOR = '#19975d'

    data = [dict(
        type = 'candlestick',
        open = df.open,
        high = df.high,
        low = df.low,
        close = df.close,
        x = pd.to_datetime(df['trade_date']),
        yaxis = 'y2',
        name = 'Candlestick',
        increasing=dict(line=dict(color=INCREASING_COLOR)),
        decreasing=dict(line=dict(color=DECREASING_COLOR)),
    )]

    layout = dict()
    fig = dict(data=data, layout=layout)

    fig['layout'] = dict()
    fig['layout']['titlefont'] = dict(size = 30)
    fig['layout']['title'] = df.name
    fig['layout']['autosize'] = True
    fig['layout']['height'] = 700
    fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
    fig['layout']['xaxis'] = dict( rangeselector = dict( visible = True ) )
    fig['layout']['yaxis'] = dict( domain = [0, 0.2], showticklabels = True )
    fig['layout']['yaxis2'] = dict( domain = [0.2, 0.9] )
    fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
    fig['layout']['margin'] = dict( t=60, b=40, r=40, l=40 )
    
    rangeselector=dict(
        visibe = True,
        x = 0, y = 0.9,
        bgcolor = 'rgba(150, 200, 250, 0.4)',
        font = dict( size = 15 ),
        buttons=list([
            dict(count=1,
                 label='reset',
                 step='all'),
            dict(count=1,
                 label='1yr',
                 step='year',
                 stepmode='backward'),
            dict(count=3,
                label='3 mo',
                step='month',
                stepmode='backward'),
            dict(count=1,
                label='1 mo',
                step='month',
                stepmode='backward'),
            dict(step='all')
        ]))
    
    fig['layout']['xaxis']['rangeselector'] = rangeselector

    colors = list(df['state'].map({True:INCREASING_COLOR,False:DECREASING_COLOR}))

    fig['data'].append( dict( x=pd.to_datetime(df['trade_date']), y=df.vol,                         
                             marker=dict( color=colors ),
                             type='bar', yaxis='y', name='vol' ) )

    name = '{}.html'.format(df.name.split('-')[1])
    py.plot( fig, filename = name , validate = False, auto_open = True )
    return df

# 画图 OHLC

In [9]:
def plotly_ohlc(code,date,period=300):
    
    df = get_one(code, date, period, print_mode=False)
    
    INCREASING_COLOR = '#e2002a'
    DECREASING_COLOR = '#19975d'
    
    trace = go.Ohlc(x=pd.to_datetime(df['trade_date']),
                open=df.open,
                high=df.high,
                low=df.low,
                close=df.close,
                yaxis = 'y2',
                name = 'OHLC',
                increasing=dict(line=dict(color=INCREASING_COLOR)),
                decreasing=dict(line=dict(color=DECREASING_COLOR)),
    )
    data = [trace]

    layout = go.Layout(
        xaxis = dict(rangeslider = dict(visible = False)),
        title = df.name,
        titlefont = dict(size = 30),
        yaxis = dict( domain = [0, 0.2], showticklabels = True),
        yaxis2 = dict( domain = [0.2, 0.97] ),
        legend = dict( orientation = 'h', y=0.97, x=0, yanchor='bottom'),
        margin = dict( t=60, b=40, r=40, l=40 ),
        plot_bgcolor = 'rgb(250, 250, 250)'
    )
    fig = dict(data=data, layout=layout)
    
    colors = list(df['state'].map({True:INCREASING_COLOR,False:DECREASING_COLOR}))

    fig['data'].append( dict( x=pd.to_datetime(df['trade_date']), y=df.vol,                         
                             marker=dict( color=colors ),
                             type='bar', yaxis='y', name='vol' ) )

    name = '{}.html'.format(df.name.split('-')[1])
    py.plot(fig, filename = name)
    return df

In [15]:
plotly_ohlc('000806.SZ',date='20190322',period=10);

# N天中M次倍量

In [10]:
def NdaysMbls(today_date,N,M,v_pct_min,v_pct_max,c_pct_min):
    all_stock = pro.stock_basic(exchange='', list_status='L', fields='ts_code,name')['ts_code']
    bl_times = dict(zip(all_stock,[0]*len(all_stock)))
    ignore = []
    for _ in range(N):
        [today,yesterday] = get_Ndays(today_date,2)
        selected_list = get_bl(today, yesterday, v_pct_min, v_pct_max, c_pct_min, ignore = ignore)
        for code in selected_list:
            bl_times[code] += 1
            if bl_times[code] == M:
                ignore.append(code)
        today_date = yesterday.name
        print('\n'+'-'*30+'\n')
    print('Done')
    return ignore

# N天中M次黄金

In [11]:
def NdaysMhjs(today_date,N,M):
    all_stock = pro.stock_basic(exchange='', list_status='L', fields='ts_code,name')['ts_code']
    hj_times = dict(zip(all_stock,[0]*len(all_stock)))
    ignore = []
    for _ in range(N):
        [d3,d2,d1] = get_Ndays(today_date,3)
        selected_list = get_hj(d1, d2, d3, ignore)
        for code in selected_list:
            hj_times[code] += 1
            if hj_times[code] == M:
                ignore.append(code)
        today_date = d2.name
        print('\n'+'-'*30+'\n')
    print('Done')
    return ignore

# N天K线图 candlestick / OHLC

In [12]:
def NK(df,N):
    def nk(df,N,start):
        df3 = pd.DataFrame(columns=['trade_date','open','high','low','close','vol','amount','state'])
        for i in range(start,len(df)-N+1,N):
            df3.loc[i+N-1,'trade_date'] = df.loc[i,'trade_date']
            df3.loc[i+N-1,'open'] = df.loc[i,'open']
            df3.loc[i+N-1,'high'] = max(df.loc[i:i+N-1,'high'])
            df3.loc[i+N-1,'low'] = min(df.loc[i:i+N-1,'low'])
            df3.loc[i+N-1,'close'] = df.loc[i+N-1,'close']
            df3.loc[i+N-1,'vol'] = sum(df.loc[i:i+N-1,'vol'])
            df3.loc[i+N-1,'amount'] = sum(df.loc[i:i+N-1,'amount'])
            df3.loc[i+N-1,'state'] = df.loc[i+N-1,'close']>df.loc[i,'open']
        df3.name = df.name
        return df3
    df_list = []
    for i in range(N):
        df_list.append(nk(df,N,start=i))
    return df_list

In [13]:
def Nplotly_candlestick(df_list):

    INCREASING_COLOR = ['#e2002a']*3#,'#f8b4b4','#ea513b']*2
    DECREASING_COLOR = ['#19975d']*3#,'#b3e0af','#92bc4e']*2
    
    data = []
    colors = []
    
    for i in range(len(df_list)):
        idict = dict(
        type = 'candlestick',
        open = df_list[i].open,
        high = df_list[i].high,
        low = df_list[i].low,
        close = df_list[i].close,
        x = pd.to_datetime(df_list[i]['trade_date']),
        yaxis = 'y2',
        name = 'GS_%s'%i,
        increasing=dict(line=dict(color=INCREASING_COLOR[i])),
        decreasing=dict(line=dict(color=DECREASING_COLOR[i])),
        )
        data.append(idict)
        colors.append(list(df_list[i]['state'].map({True:INCREASING_COLOR[i],False:DECREASING_COLOR[i]})))
        
    layout = dict()
    fig = dict(data=data, layout=layout)

    fig['layout'] = dict()
    fig['layout']['titlefont'] = dict(size = 30)
    fig['layout']['title'] = df_list[0].name
    fig['layout']['autosize'] = True
    fig['layout']['height'] = 700
    fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
    fig['layout']['xaxis'] = dict( rangeselector = dict( visible = True ) )
    fig['layout']['yaxis'] = dict( domain = [0, 0.2], showticklabels = True )
    fig['layout']['yaxis2'] = dict( domain = [0.2, 0.9] )
    fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
    fig['layout']['margin'] = dict( t=60, b=40, r=40, l=40 )

    rangeselector=dict(
        visibe = True,
        x = 0, y = 0.9,
        bgcolor = 'rgba(150, 200, 250, 0.4)',
        font = dict( size = 15 ),
        buttons=list([
            dict(count=1,
                 label='reset',
                 step='all'),
            dict(count=1,
                 label='1yr',
                 step='year',
                 stepmode='backward'),
            dict(count=3,
                label='3 mo',
                step='month',
                stepmode='backward'),
            dict(count=1,
                label='1 mo',
                step='month',
                stepmode='backward'),
            dict(step='all')
        ]))

    fig['layout']['xaxis']['rangeselector'] = rangeselector

    for i in range(len(df_list)):
        fig['data'].append(dict( x=pd.to_datetime(df_list[i]['trade_date']), y=df_list[i].vol, marker=dict( color=colors[i] ),
                            type='bar', yaxis='y', name='vol_%s'%i ))

    name = '{}.html'.format(df_list[0].name.split('-')[1])
    py.plot( fig, filename = name , validate = False, auto_open = True )

In [14]:
def Nplotly_ohlc(df_list):

    INCREASING_COLOR = ['#e2002a']*3#,'#f8b4b4','#ea513b']*2
    DECREASING_COLOR = ['#19975d']*3#,'#b3e0af','#92bc4e']*2
    
    data = []
    colors = []
    
    for i in range(len(df_list)):
        idict = go.Ohlc(
        open = df_list[i].open,
        high = df_list[i].high,
        low = df_list[i].low,
        close = df_list[i].close,
        x = pd.to_datetime(df_list[i]['trade_date']),
        yaxis = 'y2',
        name = 'OHLC_%s'%i,
        increasing=dict(line=dict(color=INCREASING_COLOR[i])),
        decreasing=dict(line=dict(color=DECREASING_COLOR[i])),
        )
        data.append(idict)
        colors.append(list(df_list[i]['state'].map({True:INCREASING_COLOR[i],False:DECREASING_COLOR[i]})))
        
    layout = go.Layout(
        xaxis = dict(rangeslider = dict(visible = False)),
        title = df_list[0].name,
        titlefont = dict(size = 30),
        yaxis = dict( domain = [0, 0.2], showticklabels = True),
        yaxis2 = dict( domain = [0.2, 0.97] ),
        legend = dict( orientation = 'h', y=0.97, x=0, yanchor='bottom'),
        margin = dict( t=60, b=40, r=40, l=40 ),
        plot_bgcolor = 'rgb(250, 250, 250)'
    )
    fig = dict(data=data, layout=layout)
    
    for i in range(len(df_list)):
        fig['data'].append(dict( x=pd.to_datetime(df_list[i]['trade_date']), y=df_list[i].vol, marker=dict( color=colors[i] ),
                            type='bar', yaxis='y', name='vol_%s'%i ))

    name = '{}.html'.format(df_list[0].name.split('-')[1])
    py.plot( fig, filename = name)

# 实测

In [127]:
today_date = '20190322'
[today,yesterday] = get_Ndays(today_date,2)

【20190322】【20190321】
获取数据共花了 0.87 秒


In [128]:
today.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,state
0,600000.SH,20190322,11.49,11.51,11.31,11.39,11.46,-0.07,-0.6108,354364.46,403972.783,False
1,600004.SH,20190322,14.40,15.00,14.31,14.89,14.43,0.46,3.1878,220079.97,324663.261,True
2,600006.SH,20190322,4.52,4.98,4.50,4.98,4.53,0.45,9.9338,606897.36,294476.945,True
3,600007.SH,20190322,14.68,14.95,14.63,14.81,14.76,0.05,0.3388,32909.36,48766.239,True
4,600008.SH,20190322,4.21,4.35,4.14,4.28,4.24,0.04,0.9434,780633.15,332835.785,True


# 筛选倍量柱

In [129]:
selected_list = get_bl(today, yesterday, v_pct_min = 1.95, v_pct_max = 2.1, c_pct_min = 1.05)

正在筛选倍量柱...

600550.SH	v:1.96	c:1.06
002163.SZ	v:2.02	c:1.08
002707.SZ	v:2.07	c:1.1
300013.SZ	v:2.08	c:1.1
300109.SZ	v:2.07	c:1.1
300758.SZ	v:2.07	c:1.08

遍历 3570 支股票共花了 17.0 秒


In [130]:
df = plotly_ohlc(selected_list[0],today_date,period=300)
df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,state
200,600550.SH,20180528,5.53,5.57,5.43,5.45,5.52,-0.07,-1.27,43865.15,24064.030,False
199,600550.SH,20180529,5.44,5.45,5.15,5.18,5.45,-0.27,-4.95,99222.09,52055.438,False
198,600550.SH,20180530,5.09,5.09,4.89,4.91,5.18,-0.27,-5.21,75397.75,37686.397,False
197,600550.SH,20180531,4.95,5.00,4.90,4.94,4.91,0.03,0.61,51087.77,25257.746,False
196,600550.SH,20180601,4.95,4.95,4.81,4.83,4.94,-0.11,-2.23,48271.43,23564.737,False


# 筛选N天M倍量柱

In [ ]:
selected_list = NdaysMbls(today_date, N=5, M=2, v_pct_min=1.9, v_pct_max=2.1, c_pct_min=1.05)

In [ ]:
df = plotly_ohlc(selected_list[0],today_date,period=300)
df.head()

# 筛选黄金柱

In [ ]:
today_date = '20190322'
[d3,d2,d1] = get_Ndays(today_date,3)
selected_list = get_hj(d1, d2, d3, ignore=[])

# 筛选N天M黄金柱

In [ ]:
selected_list = NdaysMhjs(today_date,3,2)

# N天K线图

In [ ]:
df = get_one('000568.SZ', '20190322', period=300, print_mode=True)
df.index = range(len(df))
df.head()

In [ ]:
df_list = NK(df,N=2)

In [ ]:
Nplotly_ohlc(df_list)

# 筛选缩量群

In [133]:
today_date = '20190322'
days = get_Ndays(today_date,5)
days.reverse()
selected_list = get_suo(days, rough=0.99, ignore=[])

【20190322】【20190321】【20190320】【20190319】【20190318】
获取数据共花了 1.76 秒
正在筛选 5 天缩量群...

000001.SZ
000418.SZ

遍历 3572 支股票共花了 1.0 秒 找到 2 支股票符合条件


In [135]:
df = plotly_ohlc(selected_list[1],today_date,period=300)
df.tail()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,state
4,000418.SZ,20190318,56.10,58.04,56.00,57.47,57.70,-0.23,-0.3986,82810.89,471735.013,True
3,000418.SZ,20190319,57.61,57.97,56.72,57.28,57.47,-0.19,-0.3306,40111.77,229191.646,False
2,000418.SZ,20190320,57.27,57.28,56.28,56.86,57.28,-0.42,-0.7332,34363.34,195240.590,False
1,000418.SZ,20190321,56.64,56.86,55.85,56.35,56.86,-0.51,-0.8969,29673.44,167011.625,False
0,000418.SZ,20190322,56.36,56.80,55.80,56.44,56.35,0.09,0.1597,22159.07,124615.530,True


# 汇出CSV

In [ ]:
name = pro.stock_basic(exchange='', list_status='L', fields='ts_code,name')
code = '000568.SZ'
df = get_one(code, '20190322', period=300, print_mode=True)[['trade_date','open','high','low','close','vol']]
df.index = df['trade_date']
df = df[['open','high','low','close','vol']]
df.name = '%s-%s'%(name[name['ts_code']==code]['name'].item(),code)
df.head()

In [ ]:
df['2days_p'] = (df['close']+df['open'].shift(1))/2
df['2days_v'] = (df['vol']+df['vol'].shift(1))
df.head()

In [ ]:
import numpy.random as npr
name = pro.stock_basic(exchange='', list_status='L', fields='ts_code,name')
codes = npr.choice(name['ts_code'],100)

In [ ]:
for code in codes:
    df = get_one(code, '20190322', period=300, print_mode=True)[['trade_date','open','high','low','close','vol']]
    df.index = df['trade_date']
    df = df[['open','high','low','close','vol']]
    df.name = '%s-%s'%(name[name['ts_code']==code]['name'].item(),code)
    df['2days_p'] = (df['close']+df['open'].shift(1))/2
    df['2days_v'] = df['vol']+df['vol'].shift(1)
    df['3days_p'] = (df['close']+df['open'].shift(2))/2
    df['3days_v'] = df['vol']+df['vol'].shift(1)+df['vol'].shift(2)
    df.to_csv('/Users/jackarroy/Documents/Python/FinLab/Ndays/%s.csv'%df.name)